In [1]:
import hes1_langevin_Antoine as langevin
import hes1_master_Antoine as master
import hes1_utils_Antoine as utils

import hes1_utils_general as general
import jochen_utils

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.interpolate as spinter
import scipy.fft
import time
import datetime as dt
import os

import time

import warnings
warnings.filterwarnings("ignore")

# Simulation parameters

In [2]:
alpha_m=39.93
alpha_p=21.56
mu_m=np.log(2)/30
mu_p=np.log(2)/90
h=4.78
P_0=24201.01

T=10000
delta_t=0.01

n_t=int(T/delta_t)

val_lambda=[10**x for x in np.arange(-2,2.1,0.2)]
val_Omega=[10**x for x in np.arange(-3,2.1,0.25)]
val_tau=[33]

n_lambda=np.size(val_lambda)
n_Omega=np.size(val_Omega)
n_tau=np.size(val_tau)

# Save directory

In [5]:
date_and_time=time.localtime()
string_date=str(date_and_time.tm_year)+'-'+str(date_and_time.tm_mon)+'-'+str(date_and_time.tm_mday)+'-'+str(date_and_time.tm_hour)+'-'+str(date_and_time.tm_min)+'-'+str(date_and_time.tm_sec)

read_directory = os.getcwd() + '\\paper plots ' + string_date + ' heatmap'
try:
    os.makedirs(read_directory)
    os.makedirs(read_directory + '\\plots')
    os.makedirs(read_directory + '\\data')
except FileExistsError:
    pass

# Computation

In [6]:
n_iter_master = 2
n_iter_sde = 2

T = 10000 #100000
sampling_timestep = 1
n_t=int(T/sampling_timestep)

In [7]:
arr_mean_Mm=np.zeros((n_lambda,n_Omega))
arr_mean_Pm=np.zeros((n_lambda,n_Omega))

arr_mean_Ml=np.zeros((n_lambda,n_Omega))
arr_mean_Pl=np.zeros((n_lambda,n_Omega))

In [8]:
arr_std_Mm=np.zeros((n_lambda,n_Omega))
arr_std_Pm=np.zeros((n_lambda,n_Omega))

arr_std_Ml=np.zeros((n_lambda,n_Omega))
arr_std_Pl=np.zeros((n_lambda,n_Omega))

In [9]:
arr_momFour_Mm=np.zeros((n_lambda,n_Omega))
arr_momFour_Pm=np.zeros((n_lambda,n_Omega))

arr_momFour_Ml=np.zeros((n_lambda,n_Omega))
arr_momFour_Pl=np.zeros((n_lambda,n_Omega))

In [10]:
k=0

for i in range(n_lambda-1, n_lambda):
    for j in range(n_Omega-1, n_Omega):

        valueOfLambda = val_lambda[i]
        valueOfOmega = val_Omega[j]
        valueOfTau = val_tau[k]

        t0 = time.time()

        print("Value of lambda:", valueOfLambda)
        print("Value of Omega:", valueOfOmega)
        print("Value of tau:", valueOfTau)

        if valueOfLambda <= 0.1:
            n_iter_master = 2
            n_iter_sde = 2
        else:
            n_iter_master = 2 #20
            n_iter_sde = 2 #20

        output=general.simulate_master_meanAndStd(n_iter=n_iter_master, alpha_m=alpha_m,alpha_p=alpha_p,mu_m=mu_m,mu_p=mu_p,
                                                                    h=h,
                                                                    P_0=P_0,
                                                                    lambda_s=valueOfLambda,
                                                                    tau=valueOfTau,
                                                                    P_init=0,
                                                                    M_init=0,
                                                                    T=T,
                                                                    delta_t=delta_t,
                                                                    Omega=valueOfOmega,
                                                                    sampling_timestep = sampling_timestep)

        arr_mean_Mm[i,j]=output["mean Mm"]
        arr_mean_Pm[i,j]=output["mean Pm"]
        arr_std_Mm[i,j]=output["std Mm"]
        arr_std_Pm[i,j]=output["std Pm"]
        arr_momFour_Mm[i,j]=output["momentumFour Mm"]
        arr_momFour_Pm[i,j]=output["momentumFour Pm"]

        output=general.simulate_langevin_meanAndStd(n_iter=n_iter_sde,alpha_m=alpha_m,alpha_p=alpha_p,mu_m=mu_m,mu_p=mu_p,
                                                                    h=h,
                                                                    P_0=P_0,
                                                                    lambda_s=valueOfLambda,
                                                                    tau=valueOfTau,
                                                                    P_init=0,
                                                                    M_init=0,
                                                                    T=T,
                                                                    delta_t=delta_t,
                                                                    Omega=valueOfOmega,
                                                                    sampling_timestep = sampling_timestep)            
        arr_mean_Ml[i,j]=output["mean Ml"]
        arr_mean_Pl[i,j]=output["mean Pl"]
        arr_std_Ml[i,j]=output["std Ml"]
        arr_std_Pl[i,j]=output["std Pl"]
        arr_momFour_Ml[i,j]=output["momentumFour Ml"]
        arr_momFour_Pl[i,j]=output["momentumFour Pl"]

        print("\n")
        print("Number of iterations: ", n_iter_master, n_iter_sde)
        print("Computation time: ", (time.time() - t0)//3600, "h ", ((time.time() - t0)//60)%60, "min ", (time.time() - t0)%60, "s")
        print("\n")


Value of lambda: 99.9999999999998
Value of Omega: 100.0
Value of tau: 33


Number of iterations:  2 2
Computation time:  0.0 h  2.0 min  12.425011157989502 s




In [16]:
spec= [alpha_m, alpha_p, mu_m, mu_p, h, P_0, T, delta_t]

pd.DataFrame(spec,columns=['value']).to_csv(read_directory+'\\data\\spec.csv')
pd.DataFrame(val_lambda,columns=['value']).to_csv(read_directory+'\\data\\lambda.csv')
pd.DataFrame(val_Omega,columns=['value']).to_csv(read_directory+'\\data\\Omega.csv')
pd.DataFrame(val_tau,columns=['value']).to_csv(read_directory+'\\data\\tau.csv')

pd.DataFrame(arr_std_Mm).to_csv(read_directory+'\\data\\std-Mm.csv')
pd.DataFrame(arr_std_Pm).to_csv(read_directory+'\\data\\std-Pm.csv')
pd.DataFrame(arr_std_Ml).to_csv(read_directory+'\\data\\std-Ml.csv')
pd.DataFrame(arr_std_Pl).to_csv(read_directory+'\\data\\std-Pl.csv')